# TensorFlow2 API  CIFAR-100
- Sequential API 활용
- functional
- subclassing
- gradienttape


# import library / data

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


```
Spec:
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
```


# Sequential API 활용

## model 구성

In [7]:

model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation="relu"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation="relu"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(100, activation="softmax")
    
])

## model 학습

In [8]:
# 학습 설정.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 학습
model.fit(x_train, y_train, epochs=5)


# 평가
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 6s 4ms/step - loss: 3.6843 - accuracy: 0.1428
Epoch 2/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.9790 - accuracy: 0.2694
Epoch 3/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.6672 - accuracy: 0.3328
Epoch 4/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.4623 - accuracy: 0.3732
Epoch 5/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.3041 - accuracy: 0.4068
313/313 - 0s - loss: 2.5914 - accuracy: 0.3503


[2.5914266109466553, 0.35030001401901245]

# Functional API

## model 정의

In [11]:
inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

## model 학습

In [12]:
# 모델 학습 설정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 3s 2ms/step - loss: 3.6802 - accuracy: 0.1445
Epoch 2/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.9521 - accuracy: 0.2712
Epoch 3/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.6431 - accuracy: 0.3319
Epoch 4/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.4265 - accuracy: 0.3804
Epoch 5/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.2736 - accuracy: 0.4136
313/313 - 0s - loss: 2.5754 - accuracy: 0.3570


[2.5753536224365234, 0.3569999933242798]

# subclassing

## 모델 정의

In [4]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

## 모델 학습

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 7s 4ms/step - loss: 3.6409 - accuracy: 0.1528
Epoch 2/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.9465 - accuracy: 0.2740
Epoch 3/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.6604 - accuracy: 0.3308
Epoch 4/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.4609 - accuracy: 0.3722
Epoch 5/5
1563/1563 [==============================] - 2s 1ms/step - loss: 2.3071 - accuracy: 0.4046
313/313 - 0s - loss: 2.6960 - accuracy: 0.3347


[2.6960389614105225, 0.33469998836517334]

# GradientTape 

- Forward Propagation 수행 및 중간 레이어값 저장
- Loss 값 계산
- 중간 레이어값 및 Loss를 활용한 체인룰(chain rule) 방식의 역전파(Backward Propagation) 수행
- 학습 파라미터 업데이트

```
이상 4단계로 이루어진 train_step 을 여러번 반복했습니다.

이런 과정이 TF2 API에는 model.fit()이라는 메소드 안에 모두 추상화되어 감추어져 있습니다.

Tensorflow에서 제공하는 tf.GradientTape는 위와 같이 순전파(forward pass) 로 진행된 모든 연산의 중간 레이어값을 tap에 기록하고, 이를 이용해 gradient를 계산한 후 tape를 폐기하는 기능을 수행

```

In [6]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

50000 10000


## loss / optimizer

In [7]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

## train

In [10]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            if step % batch_size == batch_size-1:
                x_batch.append(x)
                y_batch.append(y)
                loss = train_step(np.array(x_batch), np.array(y_batch))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0: last batch loss = 4.4896
Epoch 1: last batch loss = 4.3934
Epoch 2: last batch loss = 4.4065
Epoch 3: last batch loss = 4.4344
Epoch 4: last batch loss = 4.2370
It took 61.434595584869385 seconds


## evluation

In [11]:

prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 0s 690us/step


0.0313